In [2]:
import pandas_datareader.data as pdr

def get_stock_data(code):
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

TA-Libが利用できないので Candlestick を利用します
https://pypi.org/project/Candlestick/

In [ ]:
# Candlestick インストール
!pip install Candlestick

丸坊主

In [3]:
import Pattern as pattern

name = 'ガーラ'
code = 4777
df = get_stock_data(code)

# 丸坊主を検出するメソッドを実行
df = pattern.Marubozu(df)
df.tail()

,Open,High,Low,Close,Volume,Bull_Marubozu,Bear_Marubouzu
Date,,,,,,,
2023-04-18,414,424,413,417,186900,False,False
2023-04-19,415,415,407,410,184200,False,False
2023-04-20,410,410,399,402,245400,False,False
2023-04-21,402,402,392,392,285900,False,True
2023-04-24,390,400,384,389,204200,False,False


In [6]:
# 丸坊主 買い
df["bull_marubozu_text"] = df["Bull_Marubozu"].replace({ False: "", True: "買い" })
df["bull_marubozu_marker"] = df["Bull_Marubozu"].replace({ False: np.nan, True: 1 }) * df["High"]

# 丸坊主 売り
df["bear_marubozu_text"] = df["Bear_Marubouzu"].replace({ False: "", True: "売り" })
df["bear_marubozu_marker"] = df["Bear_Marubouzu"].replace({ False: np.nan, True: 1 }) * df["High"]

In [7]:
import plotly.graph_objs as go
import datetime as dt
import pandas as pd
import numpy as np

rdf = df[dt.datetime(2021,12,1):dt.datetime(2022,4,30)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

layout = { "height": 600,
           "title"  : { "text": "{} {}".format(code, name), "x":0.5 }, 
           "xaxis" : { "rangeslider": { "visible": False } },
           "yaxis": { "title": "価格（円）", "side": "left", 
                     "tickformat": "," },
          "plot_bgcolor": "light blue" }

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["bull_marubozu_marker"], 
                       mode="markers+text", 
                       text=rdf["bull_marubozu_text"],
                       textposition="top center", 
                       name="丸坊主 買い",
                       marker = { "size": 12, "color": "blue", "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
            go.Scatter(x=rdf.index, 
                       y=rdf["bear_marubozu_marker"], 
                       mode="markers+text", 
                       text=rdf["bear_marubozu_text"],
                       textposition="top center", 
                       name="丸坊主 売り",
                       marker = { "size": 12, "color": "blue", "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

包み足

In [8]:
name = '東洋製罐グループ'
code = 5901
df = get_stock_data(code)

# 包み足を検出するメソッドを実行
df = pattern.Engulf(df)

In [9]:
# 包み足 買い
df["bull_engulf_text"] = df["BullEngulf"].replace({ False: "", True: "買い" })
df["bull_engulf_marker"] = df["BullEngulf"].replace({ False: np.nan, True: 1 }) * df["High"]

# 包み足 売り
df["bear_engulf_text"] = df["BearEngulf"].replace({ False: "", True: "売り" })
df["bear_engulf_marker"] = df["BearEngulf"].replace({ False: np.nan, True: 1 }) * df["High"]

In [10]:
rdf = df[dt.datetime(2021,1,1):dt.datetime(2021,5,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

layout = { "height": 600,
           "title"  : { "text": "{} {}".format(code, name), "x":0.5 }, 
           "xaxis" : { "rangeslider": { "visible": False } },
           "yaxis": { "title": "価格（円）", "side": "left", 
                     "tickformat": "," },
          "plot_bgcolor": "light blue" }

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["bull_engulf_marker"], 
                       mode="markers+text", 
                       text=rdf["bull_engulf_text"],
                       textposition="top center", 
                       name="包み足 買い",
                       marker = { "size": 12, "color": "blue", "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
            go.Scatter(x=rdf.index, 
                       y=rdf["bear_engulf_marker"], 
                       mode="markers+text", 
                       text=rdf["bear_engulf_text"],
                       textposition="top center", 
                       name="包み足 売り",
                       marker = { "size": 12, "color": "blue", "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

はらみ足

In [12]:
name = 'オンコリスバイファーマ'
code = 4588
df = get_stock_data(code)

# はらみ足を検出するメソッドを実行
df = pattern.Harami(df)

In [13]:
# はらみ足 買い
df["bull_harami_text"] = df["BullHarami"].replace({ False: "", True: "買い" })
df["bull_harami_marker"] = df["BullHarami"].replace({ False: np.nan, True: 1 }) * df["High"]

# はらみ足 売り
df["bear_harami_text"] = df["BearHarami"].replace({ False: "", True: "売り" })
df["bear_harami_marker"] = df["BearHarami"].replace({ False: np.nan, True: 1 }) * df["High"]

In [14]:
rdf = df[dt.datetime(2022,7,1):dt.datetime(2022,11,30)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

layout = { "height": 600,
           "title"  : { "text": "{} {}".format(code, name), "x":0.5 }, 
           "xaxis" : { "rangeslider": { "visible": False } },
           "yaxis": { "title": "価格（円）", "side": "left", 
                     "tickformat": "," },
          "plot_bgcolor": "light blue" }

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["bull_harami_marker"], 
                       mode="markers+text", 
                       text=rdf["bull_harami_text"],
                       textposition="top center", 
                       name="はらみ足 買い",
                       marker = { "size": 12, "color": "blue", "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
            go.Scatter(x=rdf.index, 
                       y=rdf["bear_harami_marker"], 
                       mode="markers+text", 
                       text=rdf["bear_harami_text"],
                       textposition="top center", 
                       name="はらみ足 売り",
                       marker = { "size": 12, "color": "blue", "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

**TA-Libを利用するサンプル**

In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [ ]:
import pandas_datareader.data as pdr

def get_stock_data(code):
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

In [ ]:
import plotly.graph_objs as go
import talib as ta
import datetime as dt
import pandas as pd
import numpy as np

name = '積水ハウス'
df = get_stock_data(1928)

layout = { "height": 600,
           "title"  : { "text": "{} {}".format(code, name), "x":0.5 }, 
           "xaxis" : { "rangeslider": { "visible": False } },
           "yaxis": { "title": "価格（円）", "side": "left", 
                     "tickformat": "," },
          "plot_bgcolor": "light blue" }

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

# ローソク足
data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray")]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], 
                        x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

丸坊主

In [ ]:
marubozu = ta.CDLMARUBOZU(df["Open"], df["High"], df["Low"], df["Close"])
# チャートに表示するテキスト
df["marubozu_text"] = marubozu.replace({ 100: "買い", -100: "売り", 0: "" })

# チャートに表示するマーカーの位置を算出
df["marubozu_marker"] = (marubozu/100 * df["High"]).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["marubozu_marker"], 
                       mode="markers+text", 
                       text=rdf["marubozu_text"],
                       textposition="top center", 
                       name="丸坊主",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

寄付坊主 / 大引坊主

In [ ]:
belthold = ta.CDLBELTHOLD(df["Open"], df["High"], df["Low"], df["Close"])
df["belthold_text"] = belthold.replace({ 100: "買い", -100: "売り", 0: "" })
df["belthold_marker"] = (belthold/100 * df["High"]).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["belthold_marker"], 
                       mode="markers+text", 
                       text=rdf["belthold_text"],
                       textposition="top center", 
                       name="寄付坊主 / 大引坊主",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

包み足

In [ ]:
engulfing = ta.CDLENGULFING(df["Open"], df["High"], df["Low"], df["Close"])
df["engulfing_text"] = engulfing.replace({ 100: "買い", -100: "売り", 0: "" })
df["engulfing_marker"] = (engulfing/100 * df["High"]).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["engulfing_marker"], 
                       mode="markers+text", 
                       text=rdf["engulfing_text"],
                       textposition="top center", 
                       name="包み足",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

はらみ足

In [ ]:
harami = ta.CDLHARAMI(df["Open"], df["High"], df["Low"], df["Close"])
df["harami_text"] = harami.replace({ 100: "買い", -100: "売り", 0: "" })
df["harami_marker"] = (harami/100 * df["High"]).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["harami_marker"], 
                       mode="markers+text", 
                       text=rdf["harami_text"],
                       textposition="top center", 
                       name="はらみ足",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

包み上げ / 包み下げ

In [ ]:
outside = ta.CDL3OUTSIDE(df["Open"], df["High"], df["Low"], df["Close"])
df["outside_text"] = outside.replace({ 100: "買い", -100: "売り", 0: "" })
df["outside_marker"] = (outside/100 * df["High"]).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["outside_marker"], 
                       mode="markers+text", 
                       text=rdf["outside_text"],
                       textposition="top center", 
                       name="包み上げ / 包み下げ",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()

はらみ上げ / はらみ下げ

In [ ]:
inside = ta.CDL3INSIDE(df["Open"], df["High"], df["Low"], df["Close"])
df["inside_text"] = inside.replace({ 100: "買い", -100: "売り", 0: "" })
df["inside_marker"] = (inside * df["High"] /100).abs().replace({ 0: np.nan })

rdf = df[dt.datetime(2021,11,1):dt.datetime(2022,3,31)]
rdf.index = pd.to_datetime(rdf.index).strftime("%m-%d-%Y")

data =  [ go.Candlestick(x=rdf.index, 
                           open=rdf["Open"], high=rdf["High"], 
                           low=rdf["Low"], close=rdf["Close"],   
                           increasing_line_color="red", 
                           decreasing_line_color="gray"),
            # 検出パターン
            go.Scatter(x=rdf.index, 
                       y=rdf["inside_marker"], 
                       mode="markers+text", 
                       text=rdf["inside_text"],
                       textposition="top center", 
                       name="はらみ上げ / はらみ下げ",
                       marker = { "size": 12, "color": "blue", 
                                   "opacity": 0.6 },
                      textfont = {"size": 14, "color": "black" }),         
         ]

fig = go.Figure(layout = go.Layout(layout), data = data)
fig.update_layout({
  "xaxis":{
      "tickvals": rdf.index[::2], 
      "ticktext": ["{}-{}".format(x.split("-")[0], x.split("-")[1]) for x in rdf.index[::2]] 
      }
})
fig.show()